In [ ]:

# PASSO 1: INSTALAÇÃO E PREPARAÇÃO INICIAL

import os
import cv2
import shutil
import yaml

# Instala a biblioteca do YOLOv8
print("--- Instalando Ultralytics ---")
!pip install -q ultralytics

# Descompacta o seu dataset para uma pasta de dados "originais"
print("\n--- Descompactando o dataset original ---")
ORIGINAL_DATA_DIR = "/content/datasets_originais/"
!unzip -q baseScrewdriver.zip -d {ORIGINAL_DATA_DIR}


# PASSO 2: FUNÇÃO DE PRÉ-PROCESSAMENTO COM OPENCV

print("\n--- Definindo o pipeline de pré-processamento ---")

def preProcessDataset(base_input_dir, base_output_dir, target_size=(640, 640)):
    """
    Itera sobre um dataset no formato YOLO (train/valid/test), aplica
    um pipeline de pré-processamento OpenCV nas imagens e copia os labels.
    """
    # Itera sobre as divisões do dataset (train, valid, test)
    for split in ['train', 'valid', 'test']:
        input_images_path = os.path.join(base_input_dir, split, 'images')
        input_labels_path = os.path.join(base_input_dir, split, 'labels')

        output_images_path = os.path.join(base_output_dir, split, 'images')
        output_labels_path = os.path.join(base_output_dir, split, 'labels')

        # Cria as pastas de saída se não existirem
        os.makedirs(output_images_path, exist_ok=True)
        os.makedirs(output_labels_path, exist_ok=True)

        # 1. PRÉ-PROCESSAMENTO DAS IMAGENS
        print(f"Processando imagens em: {input_images_path}")
        if os.path.exists(input_images_path):
            for filename in os.listdir(input_images_path):
                img_path = os.path.join(input_images_path, filename)
                image = cv2.imread(img_path)
                if image is None:
                    continue

                # ETAPA 1: Redimensionamento (Padronização de tamanho)
                processed_image = cv2.resize(image, target_size, interpolation=cv2.INTER_AREA)

                # ETAPA 2: Realce de Contraste (CLAHE)
                # Converte para LAB, aplica CLAHE no canal de Luminosidade (L) e converte de volta
                lab = cv2.cvtColor(processed_image, cv2.COLOR_BGR2LAB)
                l, a, b = cv2.split(lab)
                clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
                l_clahe = clahe.apply(l)
                lab_clahe = cv2.merge((l_clahe, a, b))
                processed_image = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

                # ETAPA 3: Redução de Ruído (Suavização Gaussiana)
                processed_image = cv2.GaussianBlur(processed_image, (5, 5), 0)

                # Salva a imagem processada
                cv2.imwrite(os.path.join(output_images_path, filename), processed_image)

        # 2. CÓPIA DOS ARQUIVOS DE RÓTULO (LABELS)
        # As anotações não mudam, então apenas as copiamos.
        print(f"Copiando labels de: {input_labels_path}")
        if os.path.exists(input_labels_path):
            for filename in os.listdir(input_labels_path):
                shutil.copy(os.path.join(input_labels_path, filename), output_labels_path)


# PASSO 3: EXECUÇÃO DO PRÉ-PROCESSAMENTO

PROCESSED_DATA_DIR = "/content/datasets_processados/"
print(f"\n--- Executando pré-processamento. Saída será em: {PROCESSED_DATA_DIR} ---")

preProcessDataset(ORIGINAL_DATA_DIR, PROCESSED_DATA_DIR)

# Copia e ajusta o arquivo data.yaml para o novo diretório
original_yaml_path = os.path.join(ORIGINAL_DATA_DIR, 'data.yaml')
processed_yaml_path = os.path.join(PROCESSED_DATA_DIR, 'data.yaml')

with open(original_yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# ATUALIZA os caminhos no YAML para apontar para os dados processados!
data['path'] = PROCESSED_DATA_DIR
data['train'] = 'train/images'
data['val'] = 'valid/images'
data['test'] = 'test/images'

with open(processed_yaml_path, 'w') as f:
    yaml.dump(data, f)
print("YAML de dados processados foi criado e ajustado com sucesso!")

# PASSO 4: TREINAMENTO DO MODELO (COM DADOS PROCESSADOS)

from ultralytics import YOLO
print("\n--- Iniciando o treinamento do modelo ---")

# Carrega o modelo pré-treinado YOLOv8
model = YOLO('yolov8n.pt')

# Inicia o treinamento, apontando para o YAML dos dados JÁ PROCESSADOS
results = model.train(
    data=processed_yaml_path,
    epochs=50,
    imgsz=640,
    project='meus_treinamentos',
    name='detector_ferramentas_preprocessado_v1'
)

print("\n---Treinamento Concluído! ---")